# 🇺🇦 Ukrainian OCR Pipeline - Local Testing Demo

High-performance OCR pipeline for historical Ukrainian documents with Named Entity Recognition (NER).

**Features:**
- ⚡ GPU-accelerated TrOCR for Cyrillic handwriting (or CPU fallback)
- 🎯 Named Entity Recognition for persons and locations
- 📋 ALTO XML output for archival standards
- 🎨 Person-dense region extraction
- 📊 Progress tracking and performance monitoring

**Local Environment Requirements:**
- Python 3.8+
- PyTorch (CPU or GPU)
- Ukrainian OCR package installed

## 🚀 Setup and Installation

First, let's install the Ukrainian OCR pipeline package and check system availability.

In [ ]:
# Check if we're in the package directory and install
import os
import sys

# Add package to Python path if we're in development mode
if os.path.exists('../ukrainian_ocr'):
    sys.path.insert(0, '..')
    print("✅ Using development version from parent directory")
elif os.path.exists('./ukrainian_ocr'):
    sys.path.insert(0, '.')
    print("✅ Using development version from current directory")
else:
    print("📦 Using installed package")

# Test import
try:
    import ukrainian_ocr
    print(f"✅ Ukrainian OCR v{ukrainian_ocr.__version__} loaded successfully!")
except ImportError as e:
    print(f"❌ Error importing package: {e}")
    print("💡 Install with: pip install -e . (from package root)")

In [ ]:
# Check system capabilities
import torch
from ukrainian_ocr.utils.gpu import check_gpu_availability, optimize_for_device

print("🔍 System Information:")
print(f"  Python: {sys.version.split()[0]}")
print(f"  PyTorch: {torch.__version__}")
print(f"  CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"  GPU Memory: {gpu_memory:.1f}GB")
    recommended_device = 'cuda'
else:
    print("  Will use CPU processing (slower but works)")
    recommended_device = 'cpu'

print(f"  Recommended device: {recommended_device}")

## 📁 Setup Test Images

Set up test images from your local filesystem.

In [ ]:
# Define test image paths - modify these for your local setup
import glob
from pathlib import Path

# Option 1: Use specific test images
test_images = [
    "/home/maria/ssd990/projects/tarkovsky/Н-2982_4_1001/804-03494422-l-m-a-n2982-4-1001-00001.jpg",
    "/home/maria/ssd990/projects/tarkovsky/Н-2982_4_1001/804-03494422-l-m-a-n2982-4-1001-00002.jpg"
]

# Option 2: Auto-discover images in a directory
test_dir = "/home/maria/ssd990/projects/tarkovsky/Н-2982_4_1001/"
if os.path.exists(test_dir):
    discovered_images = glob.glob(os.path.join(test_dir, "*.jpg")) + \
                       glob.glob(os.path.join(test_dir, "*.png"))
    print(f"📁 Found {len(discovered_images)} images in {test_dir}")
    
    # Use first few for testing
    if discovered_images:
        test_images = discovered_images[:3]  # Limit to first 3 for testing

# Option 3: Manually specify your test images
# Uncomment and modify these paths:
# test_images = [
#     "/path/to/your/test/image1.jpg",
#     "/path/to/your/test/image2.jpg"
# ]

# Filter existing images
existing_images = [img for img in test_images if os.path.exists(img)]

print(f"\n📊 Test Images Setup:")
print(f"  Total specified: {len(test_images)}")
print(f"  Existing files: {len(existing_images)}")

if existing_images:
    print("\n📄 Images to process:")
    for i, img in enumerate(existing_images, 1):
        size_mb = os.path.getsize(img) / 1024 / 1024
        print(f"  {i}. {os.path.basename(img)} ({size_mb:.1f}MB)")
else:
    print("\n❌ No test images found!")
    print("💡 Please update the test_images list with valid image paths")

image_files = existing_images

## ⚙️ Configuration

Configure the OCR pipeline for local environment.

In [ ]:
from ukrainian_ocr import UkrainianOCRPipeline, OCRConfig

# Create configuration optimized for local environment
config = OCRConfig()

# Optimize based on available hardware
if recommended_device == 'cuda':
    config.device = 'cuda'
    config.batch_size = 4  # Adjust based on GPU memory
    print("🚀 Configured for GPU processing")
else:
    config.device = 'cpu'
    config.batch_size = 1  # CPU processes one at a time
    print("💻 Configured for CPU processing")

# Configure processing options
config.verbose = True  # Enable progress bars
config.post_processing.extract_person_regions = True  # Extract person-dense regions

# NER settings
config.ner.backend = 'spacy'
config.ner.confidence_threshold = 0.7

print("\n⚙️ Configuration settings:")
print(f"  Device: {config.device}")
print(f"  Batch size: {config.batch_size}")
print(f"  NER backend: {config.ner.backend}")
print(f"  Extract person regions: {config.post_processing.extract_person_regions}")
print(f"  Verbose logging: {config.verbose}")

## 🔄 Processing Pipeline

Initialize the pipeline and process your documents.

In [ ]:
# Initialize the OCR pipeline
print("🚀 Initializing Ukrainian OCR Pipeline...")
print("📥 This may take a moment on first run (downloading models)")

try:
    pipeline = UkrainianOCRPipeline(
        config=config,
        device=recommended_device,
        verbose=True
    )
    print("✅ Pipeline initialized successfully!")
    
except Exception as e:
    print(f"❌ Error initializing pipeline: {e}")
    print("💡 Make sure all dependencies are installed:")
    print("   pip install transformers kraken spacy")
    print("   python -m spacy download ru_core_news_lg")
    raise

In [ ]:
# Process images (skip if no test images)
if not image_files:
    print("⚠️ No images to process. Please set up test images in the previous cell.")
else:
    import time
    
    print(f"🔄 Processing {len(image_files)} image(s)...\n")
    
    # Create output directory
    output_dir = './test_results'
    os.makedirs(output_dir, exist_ok=True)
    
    # Start processing
    start_time = time.time()
    
    if len(image_files) == 1:
        # Single image processing
        results = [pipeline.process_single_image(
            image_files[0], 
            output_dir=output_dir,
            save_intermediate=True
        )]
    else:
        # Batch processing
        results = pipeline.process_batch(
            image_files, 
            output_dir=output_dir,
            save_intermediate=True
        )
    
    total_time = time.time() - start_time
    
    # Display results summary
    successful = sum(1 for r in results if r['success'])
    failed = len(results) - successful
    
    print(f"\n🎉 Processing complete!")
    print(f"✅ Successful: {successful}/{len(results)}")
    print(f"❌ Failed: {failed}/{len(results)}")
    print(f"⏱️ Total time: {total_time:.1f}s")
    print(f"📊 Average per image: {total_time/len(results):.1f}s")
    
    # Show pipeline statistics
    stats = pipeline.get_stats()
    print(f"\n📈 Pipeline Statistics:")
    print(f"  Images processed: {stats['images_processed']}")
    print(f"  Total processing time: {stats['total_processing_time']:.1f}s")
    print(f"  Average time per image: {stats['average_time_per_image']:.1f}s")

## 📊 Results Analysis

Analyze the processing results and view extracted entities.

In [ ]:
# Analyze results for each processed image
if 'results' in locals() and results:
    import xml.etree.ElementTree as ET
    
    for i, result in enumerate(results):
        if result['success']:
            print(f"\n📄 Image {i+1}: {os.path.basename(result['image_path'])}")
            print(f"⏱️ Processing time: {result['processing_time']:.2f}s")
            print(f"📏 Lines detected: {result['lines_detected']}")
            print(f"📝 Lines with text: {result['lines_with_text']}")
            
            # Show entities if available
            if 'entities_extracted' in result and result['entities_extracted'] > 0:
                print(f"🎯 Entities extracted: {result['total_entities']} in {result['entities_extracted']} lines")
            
            # Show output files
            print(f"\n📁 Output files:")
            for file_type, path in result['output_paths'].items():
                if path and os.path.exists(path):
                    size_kb = os.path.getsize(path) / 1024
                    print(f"  {file_type}: {os.path.basename(path)} ({size_kb:.1f}KB)")
                    
            # Try to extract sample text from ALTO
            alto_basic = result['output_paths'].get('alto_basic')
            if alto_basic and os.path.exists(alto_basic):
                try:
                    tree = ET.parse(alto_basic)
                    root = tree.getroot()
                    
                    # Extract text content from ALTO
                    ns = {'alto': 'http://www.loc.gov/standards/alto/ns-v4#'}
                    strings = root.findall('.//alto:String', ns)
                    
                    if strings:
                        print(f"\n📝 Sample extracted text (first 5 lines):")
                        for j, string_elem in enumerate(strings[:5]):
                            text = string_elem.get('CONTENT', '')
                            if text.strip():
                                print(f"  {j+1}. {text}")
                                
                except Exception as e:
                    print(f"  ⚠️ Could not parse ALTO file: {e}")
                    
        else:
            print(f"\n❌ Image {i+1} failed: {result.get('error', 'Unknown error')}")
            
        print("-" * 50)
else:
    print("⚠️ No results to analyze. Run the processing cell first.")

## 🎨 Visualization

Display processing visualizations and person-dense regions.

In [ ]:
# Display visualizations for successful results
if 'results' in locals() and results:
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    from IPython.display import display, Image as IPImage
    
    # Display visualizations for successful results
    for i, result in enumerate(results[:3]):  # Limit to first 3 images
        if result['success']:
            print(f"\n🎨 Visualizations for Image {i+1}: {os.path.basename(result['image_path'])}")
            
            # Show segmentation visualization if available
            viz_path = result['output_paths'].get('visualization')
            if viz_path and os.path.exists(viz_path):
                try:
                    plt.figure(figsize=(15, 10))
                    img = mpimg.imread(viz_path)
                    plt.imshow(img)
                    plt.title(f"Segmentation Results - {os.path.basename(result['image_path'])}")
                    plt.axis('off')
                    plt.tight_layout()
                    plt.show()
                except Exception as e:
                    print(f"  ⚠️ Could not display segmentation visualization: {e}")
                    
            # Show person region if available
            person_regions_path = result['output_paths'].get('person_regions')
            if person_regions_path and os.path.exists(person_regions_path):
                try:
                    plt.figure(figsize=(12, 8))
                    img = mpimg.imread(person_regions_path)
                    plt.imshow(img)
                    plt.title(f"Person-Dense Region - {os.path.basename(result['image_path'])}")
                    plt.axis('off')
                    plt.tight_layout()
                    plt.show()
                except Exception as e:
                    print(f"  ⚠️ Could not display person region: {e}")
            else:
                print("  ℹ️ No person-dense regions extracted")
else:
    print("⚠️ No results to visualize. Run the processing cell first.")

## 💾 Export Results

Export your processing results to different formats.

In [ ]:
# Export results summary
if 'results' in locals() and results:
    import json
    from datetime import datetime
    
    # Create comprehensive results summary
    summary = {
        'processing_info': {
            'timestamp': datetime.now().isoformat(),
            'total_images': len(results),
            'successful': sum(1 for r in results if r['success']),
            'failed': sum(1 for r in results if not r['success']),
            'total_processing_time': sum(r.get('processing_time', 0) for r in results),
            'system_info': {
                'device_used': config.device,
                'batch_size': config.batch_size,
                'python_version': sys.version.split()[0],
                'pytorch_version': torch.__version__
            }
        },
        'results': results
    }
    
    # Save results summary as JSON
    summary_path = os.path.join(output_dir, 'processing_summary.json')
    with open(summary_path, 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2, ensure_ascii=False, default=str)
    
    # Create text summary
    text_summary_path = os.path.join(output_dir, 'processing_summary.txt')
    with open(text_summary_path, 'w', encoding='utf-8') as f:
        f.write("Ukrainian OCR Pipeline - Processing Summary\n")
        f.write("=" * 45 + "\n\n")
        f.write(f"Timestamp: {summary['processing_info']['timestamp']}\n")
        f.write(f"Total Images: {summary['processing_info']['total_images']}\n")
        f.write(f"Successful: {summary['processing_info']['successful']}\n")
        f.write(f"Failed: {summary['processing_info']['failed']}\n")
        f.write(f"Total Time: {summary['processing_info']['total_processing_time']:.2f}s\n")
        f.write(f"Device Used: {summary['processing_info']['system_info']['device_used']}\n\n")
        
        # Add details for each image
        for i, result in enumerate(results):
            f.write(f"Image {i+1}: {os.path.basename(result['image_path'])}\n")
            if result['success']:
                f.write(f"  Status: Success\n")
                f.write(f"  Processing Time: {result['processing_time']:.2f}s\n")
                f.write(f"  Lines Detected: {result['lines_detected']}\n")
                f.write(f"  Lines with Text: {result['lines_with_text']}\n")
                if 'entities_extracted' in result:
                    f.write(f"  Entities Extracted: {result.get('total_entities', 0)}\n")
            else:
                f.write(f"  Status: Failed\n")
                f.write(f"  Error: {result.get('error', 'Unknown error')}\n")
            f.write("\n")
    
    print(f"📄 Results exported:")
    print(f"  JSON summary: {summary_path}")
    print(f"  Text summary: {text_summary_path}")
    
    # Show output directory contents
    print(f"\n📁 Output directory ({output_dir}):")
    for root, dirs, files in os.walk(output_dir):
        level = root.replace(output_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            file_path = os.path.join(root, file)
            size_kb = os.path.getsize(file_path) / 1024
            print(f"{subindent}{file} ({size_kb:.1f}KB)")
    
else:
    print("⚠️ No results to export. Run the processing cell first.")

## 🧹 Cleanup

Clean up GPU memory and temporary resources.

In [ ]:
# Clean up pipeline resources
if 'pipeline' in locals():
    pipeline.cleanup()
    print("✅ Pipeline resources cleaned up")

# Show final memory usage
if torch.cuda.is_available():
    print(f"\n📊 Final GPU Memory Usage:")
    allocated = torch.cuda.memory_allocated() / 1024**3
    cached = torch.cuda.memory_reserved() / 1024**3
    print(f"  Allocated: {allocated:.2f}GB")
    print(f"  Cached: {cached:.2f}GB")
    
    # Force cleanup
    torch.cuda.empty_cache()
    print("  ✅ GPU cache cleared")

print("\n🎉 Local testing complete!")
if 'output_dir' in locals():
    print(f"📚 Check the results in: {os.path.abspath(output_dir)}")

## 🔧 Development & Testing Utilities

Additional utilities for development and testing.

In [ ]:
# Quick model testing function
def test_individual_components():
    """Test individual pipeline components"""
    print("🔧 Testing individual components...\n")
    
    # Test segmentation
    try:
        from ukrainian_ocr.core.segmentation import KrakenSegmenter
        import cv2
        
        if image_files:
            test_img = cv2.imread(image_files[0])
            segmenter = KrakenSegmenter(device='cpu')  # Use CPU for testing
            lines = segmenter.segment_image(test_img)
            print(f"✅ Segmentation: {len(lines)} lines detected")
        else:
            print("⚠️ Segmentation: No test images available")
    except Exception as e:
        print(f"❌ Segmentation error: {e}")
    
    # Test OCR
    try:
        from ukrainian_ocr.core.ocr import TrOCRProcessor
        import numpy as np
        
        ocr_processor = TrOCRProcessor(device='cpu')  # Use CPU for testing
        
        # Create a small test image
        test_image = np.ones((50, 200, 3), dtype=np.uint8) * 255
        result = ocr_processor.recognize_text(test_image)
        print(f"✅ OCR: Model loaded, test returned: '{result['text']}'")
    except Exception as e:
        print(f"❌ OCR error: {e}")
    
    # Test NER
    try:
        from ukrainian_ocr.core.ner import NERExtractor
        
        ner_extractor = NERExtractor()
        entities = ner_extractor.extract_entities("Иван Петров из Киева")
        print(f"✅ NER: {len(entities)} entities found in test text")
    except Exception as e:
        print(f"❌ NER error: {e}")

# Uncomment to test individual components
# test_individual_components()

In [ ]:
# Performance benchmarking
def benchmark_pipeline():
    """Benchmark pipeline performance on different settings"""
    print("📊 Benchmarking pipeline performance...\n")
    
    if not image_files:
        print("⚠️ No test images available for benchmarking")
        return
    
    test_image = image_files[0]
    
    # Test different configurations
    configs_to_test = [
        {'device': 'cpu', 'batch_size': 1, 'name': 'CPU Single'},
    ]
    
    if torch.cuda.is_available():
        configs_to_test.extend([
            {'device': 'cuda', 'batch_size': 1, 'name': 'GPU Single'},
            {'device': 'cuda', 'batch_size': 4, 'name': 'GPU Batch'},
        ])
    
    for test_config in configs_to_test:
        try:
            print(f"Testing {test_config['name']}...")
            
            # Create test pipeline
            test_pipeline = UkrainianOCRPipeline(
                device=test_config['device'],
                batch_size=test_config['batch_size'],
                verbose=False
            )
            
            # Time the processing
            import time
            start = time.time()
            result = test_pipeline.process_single_image(
                test_image,
                output_dir='./benchmark_temp',
                save_intermediate=False
            )
            end = time.time()
            
            if result['success']:
                print(f"  ✅ {test_config['name']}: {end-start:.2f}s")
            else:
                print(f"  ❌ {test_config['name']}: Failed")
            
            # Cleanup
            test_pipeline.cleanup()
            
        except Exception as e:
            print(f"  ❌ {test_config['name']}: Error - {e}")
    
    # Cleanup benchmark temp directory
    import shutil
    if os.path.exists('./benchmark_temp'):
        shutil.rmtree('./benchmark_temp')

# Uncomment to run benchmark
# benchmark_pipeline()

## 🚀 Next Steps

**What you can do with the results:**

1. **ALTO XML files** - Import into eScriptorium or other document analysis tools
2. **Enhanced ALTO** - Contains named entity annotations for persons and locations
3. **Person-dense regions** - Cropped images focusing on genealogically valuable content
4. **Entity extraction** - Use the identified persons and locations for genealogical research

**For production use:**
- Use the CLI interface: `ukrainian-ocr --help`
- Customize configuration files
- Integrate with existing workflows
- Process larger batches of documents

**Development:**
- Modify component implementations in `ukrainian_ocr/core/`
- Add custom NER models
- Extend ALTO XML generation
- Add new post-processing features

**Need help?**
- Check the package documentation
- Review the configuration options
- Test individual components using the utilities above
